In [16]:
!pip3 install catboost

Defaulting to user installation because normal site-packages is not writeable


In [19]:
import pandas as pd
import pickle


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier
from sklearn.metrics import(
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

In [20]:
train = pd.read_csv("data_rrg_big.csv")

In [21]:
train = train.dropna()

In [22]:
train.head(5)

,id,rrg_mode_лежа,rrg_mode_стоя,rrg_mean_лежа,rrg_mean_стоя,rrg_median_лежа,rrg_median_стоя,rrg_std_лежа,rrg_std_стоя,rrg_min_лежа,...,rrg_trim_mean_стоя,rrg_iqr_лежа,rrg_iqr_стоя,rrg_mad_лежа,rrg_mad_стоя,rrg_cv_лежа,rrg_cv_стоя,rrg_rms_лежа,rrg_rms_стоя,patient
0,АГ-001_1,66.079295,71.090047,65.0,70.0,65.0,70.0,1.0,2.0,63.291139,...,70.378093,1.436616,2.645911,0.854409,1.347800,2.259452,3.462479,0.887758,2.072748,AG
1,АГ-002_1,81.521739,90.361446,82.0,91.0,82.0,90.0,3.0,4.0,70.093458,...,90.993558,4.531859,5.514706,2.203679,2.840909,4.393952,4.553570,1.833997,1.464579,AG
2,АГ-003_1,64.655172,87.209302,63.0,85.0,63.0,85.0,2.0,5.0,55.350554,...,85.526112,3.459655,6.790183,1.643926,3.260060,4.138637,5.842208,2.877912,4.412065,AG
3,АГ-004_1,50.000000,66.371681,51.0,67.0,50.0,67.0,3.0,4.0,46.153846,...,67.471684,3.661428,5.438286,1.656068,2.736747,6.548297,6.804123,2.491457,1.964167,AG
4,АГ-005_1,81.967213,88.757396,82.0,90.0,82.0,90.0,3.0,4.0,67.873303,...,90.535760,4.055759,4.932408,1.852081,2.231147,3.689856,4.748197,1.617673,3.223781,AG


In [37]:
X = train[list(train.columns[1:-1])]
y = train[["patient"]]

In [38]:
X = X.astype(int)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=0xDD,
    test_size=0.5,
    shuffle=True
)

In [ ]:
model = CatBoostClassifier()

param_grid = {
    'depth': list(range(2, 12, 2)),
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': list(range(30, 80, 10)),
    'l2_leaf_reg': list(range(1, 7, 2)),
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

In [40]:
rf_model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)

rf_model.fit(X_train, y_train)

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=5, n_estimators=50, random_state=42)

In [41]:
# Прогнозирование на тестовых данных
y_pred = rf_model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вывод отчета о классификации
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Матрица ошибок
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.69
Classification Report:
              precision    recall  f1-score   support

          AG       0.70      0.86      0.77        35
           H       0.50      0.38      0.43         8
         IM6       0.80      0.50      0.62        16

    accuracy                           0.69        59
   macro avg       0.67      0.58      0.60        59
weighted avg       0.70      0.69      0.68        59

Confusion Matrix:
[[30  3  2]
 [ 5  3  0]
 [ 8  0  8]]


In [29]:
boost_model = CatBoostClassifier(
    random_seed=45,
    depth=10,
    iterations=60,
    learning_rate=0.5,
    cat_features=["rrg_iqr_стоя", "rrg_max_лежа", "rrg_max_стоя",
                  "rrg_iqr_лежа"]
)
boost_model.fit(X_train, y_train)


0:	learn: 0.9505051	total: 145ms	remaining: 8.54s
1:	learn: 0.8464557	total: 239ms	remaining: 6.95s
2:	learn: 0.7544232	total: 402ms	remaining: 7.64s
3:	learn: 0.6884230	total: 473ms	remaining: 6.63s
4:	learn: 0.6033083	total: 574ms	remaining: 6.31s
5:	learn: 0.5555896	total: 749ms	remaining: 6.74s
6:	learn: 0.5146001	total: 858ms	remaining: 6.49s
7:	learn: 0.4795048	total: 954ms	remaining: 6.2s
8:	learn: 0.4363923	total: 1.05s	remaining: 5.98s
9:	learn: 0.4015173	total: 1.23s	remaining: 6.13s
10:	learn: 0.3778430	total: 1.36s	remaining: 6.06s
11:	learn: 0.3508311	total: 1.49s	remaining: 5.96s
12:	learn: 0.3301034	total: 1.68s	remaining: 6.08s
13:	learn: 0.3112487	total: 1.81s	remaining: 5.94s
14:	learn: 0.2907766	total: 2s	remaining: 5.99s
15:	learn: 0.2761234	total: 2.18s	remaining: 6.01s
16:	learn: 0.2607399	total: 2.31s	remaining: 5.86s
17:	learn: 0.2458174	total: 2.52s	remaining: 5.88s
18:	learn: 0.2334293	total: 2.7s	remaining: 5.83s
19:	learn: 0.2194005	total: 2.83s	remaining: 5

In [30]:
# Получение важности признаков
feature_importances = boost_model.get_feature_importance()
for feature, importance in zip(X_train.columns, feature_importances):
    print(f'{feature}: {importance}')

rrg_mode_лежа: 5.173630722807106
rrg_mode_стоя: 7.644499081849398
rrg_mean_лежа: 1.7691169673649783
rrg_mean_стоя: 5.757771762559705
rrg_median_лежа: 6.3881942477172675
rrg_median_стоя: 1.433496479719989
rrg_std_лежа: 0.4677777047375813
rrg_std_стоя: 5.577890989010297
rrg_min_лежа: 6.304307927835148
rrg_min_стоя: 4.084667149256705
rrg_max_лежа: 0.0
rrg_max_стоя: 0.0
rrg_trim_mean_лежа: 3.660053284038261
rrg_trim_mean_стоя: 1.0638981961559395
rrg_iqr_лежа: 6.789642085796901
rrg_iqr_стоя: 23.550479116391752
rrg_mad_лежа: 1.9725731581186128
rrg_mad_стоя: 2.8463269201076193
rrg_cv_лежа: 1.701786032161632
rrg_cv_стоя: 1.9633301929834521
rrg_rms_лежа: 8.837415568546735
rrg_rms_стоя: 3.013142412840904


In [31]:
y_pred = boost_model.predict(X_test)

In [32]:
# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

F1 = f1_score(y_pred=y_pred, y_true=y_test, average='micro')
print(f"F1: {F1:.2f}")

# Вывод отчета о классификации
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Матрица ошибок
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.78
F1: 0.78
Classification Report:
              precision    recall  f1-score   support

          AG       0.76      0.91      0.83        35
           H       0.67      0.25      0.36         8
         IM6       0.86      0.75      0.80        16

    accuracy                           0.78        59
   macro avg       0.76      0.64      0.66        59
weighted avg       0.77      0.78      0.76        59

Confusion Matrix:
[[32  1  2]
 [ 6  2  0]
 [ 4  0 12]]


In [34]:
filename = 'model_rr.pk'
with open(filename, 'wb') as file:
  pickle.dump(boost_model, file)